In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [6]:
from dotenv import load_dotenv
load_dotenv("../.env")

False

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [7]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    allow_delegation=True,
    verbose=True,
    tools=[search_tool, scrape_tool]
)

In [8]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    allow_delegation=True,
    tools=[search_tool, scrape_tool],
    verbose=True
)

In [9]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    allow_delegation=True,
    tools=[search_tool, scrape_tool],
    verbose=True
)

In [10]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    allow_delegation=True,
    tools=[search_tool, scrape_tool],
    verbose=True
)

In [11]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [12]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

In [13]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)

In [14]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

In [15]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# define the crew with agent and tasks
financial_trading_crew = Crew (
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    verbose=True,
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
)

In [16]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [17]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

# Agent: Crew Manager
## Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


# Agent: Crew Manager
## Thought: To effectively monitor and analyze market data for AAPL, I will need to gather detailed information on market trends and movements related to this stock. It's essential to use statistical modeling and machine learning techniques to identify patterns and predict potential opportunities or threats in the market.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"AAPL stock market analysis\"}"
## Tool Output: 
{'searchParameters': {'q': 'AAPL stock market analysis', 'type': 'search', 'num': 10, 'engine': 'google'}, 'knowledgeGraph': {'title': 'Apple', 'type': 'Technology company', 'website': '', 'imageUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQwGQRv5TjjkycpctY66mOg_e2-npacrmjAb6_jAWhzlzkFE3OTjxyzbA&s=0', 'descripti

In [18]:
from IPython.display import Markdown
Markdown(result.raw)

After conducting comprehensive research on trading AAPL stocks CFDs, it is important to highlight some of the key risks associated with this trading activity. One significant risk is the high volatility of the stock market, which can lead to rapid price fluctuations and potential losses for traders. Additionally, trading with leverage, as common in CFD trading, increases the risk of amplified losses if the market moves against the trader.

Furthermore, Apple's stock price is heavily influenced by external factors such as economic indicators, product launches, and market sentiment. Failure to accurately predict these factors can result in losses for traders. Moreover, the complexity of options trading and CFDs poses a risk for inexperienced traders who may not fully understand the intricacies of these trading instruments.

To mitigate these risks, it is crucial for traders to implement effective risk management strategies such as setting stop-loss orders to limit potential losses, utilizing guaranteed stop orders to protect against significant market movements, and staying informed about market trends and company developments.

Overall, while trading AAPL stocks CFDs can present lucrative opportunities, traders must proceed with caution, conduct thorough market analysis, and employ sound risk management practices to mitigate potential risks and maximize trading outcomes.